In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
import math

In [17]:
def make_data(directory):
    files = os.listdir(directory)
    d = {}
    for file in files:
        file_name = os.path.basename(file)
        os.chdir(directory)
        f = open(file_name)
        file_content = f.readlines()[:-1]
        for i in file_content:
            l = i.strip().split()
            if len(l)>2:
                k = ' '.join(l[:-1])
                v = l[-1]
            else:
                k = l[0]
                v = l[-1]
#             else:
#                 print(i,"n")
#                 k,v = i.strip().split()
#                 print(k,v)
            if k in d:
                d[k].append(v)
            else:
                d[k]=[v]
                
    return d        
m = make_data(r"C:\Users\admin\Downloads\dataset\train")
n = make_data(r"C:\Users\admin\Downloads\dataset\test")  

In [2]:
train_data = pd.DataFrame(m)
test_data = pd.DataFrame(n)
train_data.to_csv(r"C:\Users\admin\Downloads\dataset\train.csv",index=False)
test_data.to_csv(r"C:\Users\admin\Downloads\dataset\test.csv",index=False)

NameError: name 'm' is not defined

In [188]:
train_data = pd.read_csv(r"C:\Users\admin\Downloads\dataset\train.csv")
test_data = pd.read_csv(r"C:\Users\admin\Downloads\dataset\test.csv")

In [189]:
id_ = test_data['ID']

In [190]:
ids = list(train_data['Delivery_person_ID'])
te_ids = list(test_data['Delivery_person_ID'])
count=0
for id in te_ids:
    if id in ids:
        count+=1

In [191]:
test_data.shape,count

((11399, 19), 11399)

In [192]:
for col in train_data.columns:
    print(col,len(train_data[col].unique()))

ID 45592
Delivery_person_ID 1320
Delivery_person_Age 23
Delivery_person_Ratings 29
Restaurant_latitude 657
Restaurant_longitude 518
Delivery_location_latitude 4373
Delivery_location_longitude 4373
Order_Date 44
Time_Orderd 177
Time_Order_picked 193
Weather conditions 7
Road_traffic_density 5
Vehicle_condition 4
Type_of_order 4
Type_of_vehicle 4
multiple_deliveries 5
Festival 3
City 4
Time_taken (min) 45


In [193]:
def to_numeric(df,is_train=None):
    bins= [18,25,30,40,50,100]
    df = df.replace('NaN',None)
    numeric_columns = ['Delivery_person_Age',
           'Delivery_person_Ratings', 'Restaurant_latitude',
           'Restaurant_longitude', 'Delivery_location_latitude',
           'Delivery_location_longitude',
        'Vehicle_condition','multiple_deliveries']
    for col in numeric_columns:
        df[col] = df[col].astype(np.number)
    df['Order_Date'] = pd.to_datetime(df['Order_Date'])
    df['Order_month'] = df['Order_Date'].dt.month
    df['Order_day'] = df['Order_Date'].dt.day
    df['Order_weekday'] = df['Order_Date'].dt.weekday
    df["Is_weekend"] = np.where((df["Order_weekday"] >4),1,0)
    df["Is_mon"] = np.where(df["Order_weekday"] ==1,1,0)
    temp = df['Time_Orderd'].str.split(':', expand=True)
    df['Order_hrs'],df['Order_mins'] = temp[0],temp[1]
    temp=df['Time_Order_picked'].str.split(':', expand=True)
    df['Order_p_hrs'],df['Order_p_mins'] = temp[0],temp[1]
    df['city'] = df['Delivery_person_ID'].str[:4]   
    dcols = ['Order_hrs','Order_mins','Order_p_hrs','Order_p_mins']
    df['distance'] = (df['Restaurant_latitude']-df['Delivery_location_latitude'])**2 + (df['Restaurant_longitude']-df['Delivery_location_longitude'])**2
    if is_train:
#         df['distance'] = (df['Restaurant_latitude']-df['Delivery_location_latitude'])**2 + (df['Restaurant_longitude']-df['Delivery_location_longitude'])**2
        df['time/distance'] = df["Time_taken (min)"].astype(np.number)/df['distance']
        df_temp = df.groupby('Delivery_person_ID')["time/distance"].mean()
        df = df.merge(df_temp, on='Delivery_person_ID',how='left')
        df = df.dropna()
        df = df.drop('time/distance_x',axis=1,)

    for col in dcols:
        df[col] = df[col].replace('NaN',0)
        df = df.fillna(0)
        df[col] = df[col].apply(int)
    labels = ['NAdult','YAdult','OldAdult','VoldAdult','VVoldAdult']
    df['AgeGroup'] = pd.cut(df['Delivery_person_Age'], bins=bins, labels=labels, right=False)
    cat_cols = ["Weather conditions", "Road_traffic_density", "Type_of_order", "Type_of_vehicle", "Festival","city","AgeGroup"]
    for col in cat_cols:
        df[col] = df[col].astype('category')
    
    

    df = df.drop(["Time_Orderd",'Time_Order_picked','ID','Order_Date','City'],axis=1,)

    return df
train_data = to_numeric(train_data,is_train=True)
test_data = to_numeric(test_data)

In [194]:
test_data.shape

(11399, 26)

In [195]:
test_data = test_data.merge(train_data[['Delivery_person_ID','time/distance_y']].drop_duplicates('Delivery_person_ID',keep='last'),on='Delivery_person_ID',how='left')

In [196]:
target_y = train_data['Time_taken (min)']
train_data.drop(['Time_taken (min)','Delivery_person_ID'],axis=1,inplace=True)
test_data.drop('Delivery_person_ID',axis=1,inplace=True)

In [197]:
train_data.shape,test_data.shape

((41367, 26), (11399, 26))

In [198]:
all_data = pd.concat([train_data,test_data])
all_data.shape

(52766, 26)

In [199]:
numeric_columns = ['Delivery_person_Age',
           'Delivery_person_Ratings', 'Restaurant_latitude',
           'Restaurant_longitude', 'Delivery_location_latitude',
           'Delivery_location_longitude',
        'Vehicle_condition','multiple_deliveries','time/distance_y','distance']


In [200]:
from scipy.stats import norm,skew
skewed_feats = all_data[numeric_columns].apply(lambda x:skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({"Skew":skewed_feats})

In [201]:
skewed_feats

distance                       32.616312
time/distance_y                 0.722909
multiple_deliveries             0.329884
Vehicle_condition               0.019327
Delivery_location_latitude     -0.702351
Delivery_person_Age            -0.801478
Restaurant_latitude            -1.144903
Delivery_location_longitude    -2.973118
Restaurant_longitude           -3.060907
Delivery_person_Ratings        -5.763150
dtype: float64

In [204]:
skewness = skewness[abs(skewness)>0.75]
from scipy.special import boxcox1p
skewed_features = skewness.index
lam =0.15
for feat in skewed_features:
    all_data[feat] = boxcox1p(all_data[feat],lam)
cat_cols = ["Weather conditions", "Road_traffic_density", "Type_of_order", "Type_of_vehicle", "Festival","city","AgeGroup"]
for col in cat_cols:
    all_data[col] = all_data[col].astype('category') 
all_data = pd.get_dummies(all_data)
print(all_data.shape)
train_data = all_data.iloc[:41367]
test_data = all_data.iloc[41367:]

(52766, 69)


In [205]:
train_data.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'Order_month', 'Order_day', 'Order_weekday',
       'Is_weekend', 'Is_mon', 'Order_hrs', 'Order_mins', 'Order_p_hrs',
       'Order_p_mins', 'distance', 'time/distance_y', 'Weather conditions_0',
       'Weather conditions_Cloudy', 'Weather conditions_Fog',
       'Weather conditions_Sandstorms', 'Weather conditions_Stormy',
       'Weather conditions_Sunny', 'Weather conditions_Windy',
       'Road_traffic_density_0', 'Road_traffic_density_High',
       'Road_traffic_density_Jam', 'Road_traffic_density_Low',
       'Road_traffic_density_Medium', 'Type_of_order_Buffet',
       'Type_of_order_Drinks', 'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_bicycle', 'Type_of_vehicle_electric_scooter',
       'Type_of_vehicle_motorcycle', 'Type_of_v

In [173]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import 
# poly=PolynomialFeatures(3)
# num_train_data = poly.fit_transform(train_data[numeric_columns])
# num_test_data = poly.transform(test_data[numeric_columns])

SyntaxError: invalid syntax (<ipython-input-173-a2303ce4ebe8>, line 3)

In [174]:
num_train_data.shape

(41367, 220)

In [207]:
unnum_cols = list(set(train_data.columns) - set(numeric_columns))

In [208]:
pd.DataFrame(num_train_data).shape,train_data[unnum_cols].shape

((41367, 220), (41367, 59))

In [209]:
t_data = np.concatenate([num_train_data,train_data[unnum_cols].to_numpy()],axis=1)
te_data = np.concatenate([num_test_data,test_data[unnum_cols].to_numpy()],axis=1)


In [210]:
num_test_data.shape

(11399, 220)

In [211]:
from sklearn.model_selection import train_test_split
tr,te,ty,tey = train_test_split(t_data, target_y, test_size=0.30, random_state=42) 

In [212]:
pred.shape

(11399,)

In [213]:
te_data.shape

(11399, 279)

In [221]:
!pip install lightgbm

In [234]:
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
lgb=lgbm.LGBMRegressor(objective='regression')
params_grid = {'n_estimators':range(200, 800, 100), 'num_leaves':range(5,60,10),'learning_rate':[0.05]}
clf = GridSearchCV(lgb, params_grid, cv=2)
clf.fit(tr, ty)

GridSearchCV(cv=2, estimator=LGBMRegressor(objective='regression'),
             param_grid={'learning_rate': [0.05],
                         'n_estimators': range(200, 800, 100),
                         'num_leaves': range(5, 60, 10)})

In [235]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
predv = clf.predict(te)
rmse = (np.sqrt(mean_squared_error(tey, predv)))
r2 = r2_score(tey, predv)
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))
predt = clf.predict(te_data)

RMSE: 4.18
R2: 0.80


In [237]:
clf.best_params_

{'learning_rate': 0.05, 'n_estimators': 300, 'num_leaves': 55}

In [240]:
#cat
params: {'depth': 5,
  'l2_leaf_reg': 1,
  'iterations': 500,
  'learning_rate': 0.05}
         
lgb_params = {'learning_rate': 0.05, 'n_estimators': 300, 'num_leaves': 55}

In [245]:
import lightgbm as lgbm
lgb_wh=lgbm.LGBMRegressor(objective='regression',learning_rate= 0.05, n_estimators=300, num_leaves=55)
lgb_wh.fit(t_data, target_y)

LGBMRegressor(learning_rate=0.05, n_estimators=300, num_leaves=55,
              objective='regression')

In [253]:
predtlgb = lgb_wh.predict(te_data)

In [251]:
import catboost as cb
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=500,depth=5,learning_rate=0.05,l2_leaf_reg= 1)
# grid = {'iterations': [500,600],
#          'learning_rate': [0.05],
#          'depth': [5,8,10],
#          'l2_leaf_reg': [ 0.5, 1,2]}
model.fit(t_data, target_y)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 9.0781957	total: 65.7ms	remaining: 32.8s
1:	learn: 8.8417912	total: 124ms	remaining: 31s
2:	learn: 8.6059031	total: 187ms	remaining: 31s
3:	learn: 8.4007779	total: 246ms	remaining: 30.5s
4:	learn: 8.2120453	total: 312ms	remaining: 30.8s
5:	learn: 8.0335405	total: 371ms	remaining: 30.5s
6:	learn: 7.8596281	total: 425ms	remaining: 29.9s
7:	learn: 7.7073321	total: 493ms	remaining: 30.3s
8:	learn: 7.5639367	total: 560ms	remaining: 30.5s
9:	learn: 7.4321621	total: 621ms	remaining: 30.4s
10:	learn: 7.2953750	total: 689ms	remaining: 30.6s
11:	learn: 7.1695155	total: 750ms	remaining: 30.5s
12:	learn: 7.0526369	total: 813ms	remaining: 30.5s
13:	learn: 6.9371454	total: 873ms	remaining: 30.3s
14:	learn: 6.8365444	total: 934ms	remaining: 30.2s
15:	learn: 6.7449381	total: 993ms	remaining: 30s
16:	learn: 6.6584275	total: 1.05s	remaining: 30s
17:	learn: 6.5743788	total: 1.14s	remaining: 30.4s
18:	learn: 6.4988908	total: 1.24s	remaining: 31.3s
19:	learn: 6.4189936	total: 1.32s	remaining: 31.

162:	learn: 4.7590117	total: 10.1s	remaining: 21s
163:	learn: 4.7575790	total: 10.2s	remaining: 20.9s
164:	learn: 4.7540775	total: 10.3s	remaining: 20.8s
165:	learn: 4.7523580	total: 10.3s	remaining: 20.8s
166:	learn: 4.7516455	total: 10.5s	remaining: 20.8s
167:	learn: 4.7500012	total: 10.6s	remaining: 20.9s
168:	learn: 4.7483798	total: 10.6s	remaining: 20.8s
169:	learn: 4.7445547	total: 10.7s	remaining: 20.7s
170:	learn: 4.7405702	total: 10.7s	remaining: 20.6s
171:	learn: 4.7394189	total: 10.8s	remaining: 20.6s
172:	learn: 4.7387935	total: 10.9s	remaining: 20.6s
173:	learn: 4.7381027	total: 11s	remaining: 20.6s
174:	learn: 4.7352677	total: 11.1s	remaining: 20.5s
175:	learn: 4.7326870	total: 11.1s	remaining: 20.5s
176:	learn: 4.7320185	total: 11.2s	remaining: 20.4s
177:	learn: 4.7309633	total: 11.2s	remaining: 20.3s
178:	learn: 4.7300528	total: 11.3s	remaining: 20.3s
179:	learn: 4.7289150	total: 11.4s	remaining: 20.2s
180:	learn: 4.7269673	total: 11.4s	remaining: 20.1s
181:	learn: 4.72

324:	learn: 4.4717140	total: 20.5s	remaining: 11s
325:	learn: 4.4711779	total: 20.5s	remaining: 11s
326:	learn: 4.4685291	total: 20.6s	remaining: 10.9s
327:	learn: 4.4674903	total: 20.7s	remaining: 10.8s
328:	learn: 4.4661787	total: 20.7s	remaining: 10.8s
329:	learn: 4.4651951	total: 20.8s	remaining: 10.7s
330:	learn: 4.4642105	total: 20.8s	remaining: 10.6s
331:	learn: 4.4630002	total: 20.9s	remaining: 10.6s
332:	learn: 4.4622121	total: 20.9s	remaining: 10.5s
333:	learn: 4.4609548	total: 21s	remaining: 10.4s
334:	learn: 4.4601753	total: 21.1s	remaining: 10.4s
335:	learn: 4.4593815	total: 21.1s	remaining: 10.3s
336:	learn: 4.4581595	total: 21.2s	remaining: 10.3s
337:	learn: 4.4572333	total: 21.3s	remaining: 10.2s
338:	learn: 4.4561639	total: 21.3s	remaining: 10.1s
339:	learn: 4.4543816	total: 21.4s	remaining: 10.1s
340:	learn: 4.4535187	total: 21.4s	remaining: 9.99s
341:	learn: 4.4525984	total: 21.5s	remaining: 9.93s
342:	learn: 4.4521178	total: 21.5s	remaining: 9.86s
343:	learn: 4.4508

485:	learn: 4.3054139	total: 30.6s	remaining: 883ms
486:	learn: 4.3048598	total: 30.7s	remaining: 820ms
487:	learn: 4.3042134	total: 30.8s	remaining: 757ms
488:	learn: 4.3030095	total: 30.8s	remaining: 694ms
489:	learn: 4.3018463	total: 30.9s	remaining: 631ms
490:	learn: 4.3013016	total: 30.9s	remaining: 567ms
491:	learn: 4.3009019	total: 31s	remaining: 504ms
492:	learn: 4.2998660	total: 31.1s	remaining: 441ms
493:	learn: 4.2991398	total: 31.1s	remaining: 378ms
494:	learn: 4.2980747	total: 31.2s	remaining: 315ms
495:	learn: 4.2974536	total: 31.2s	remaining: 252ms
496:	learn: 4.2970867	total: 31.3s	remaining: 189ms
497:	learn: 4.2966561	total: 31.4s	remaining: 126ms
498:	learn: 4.2953128	total: 31.4s	remaining: 63ms
499:	learn: 4.2947042	total: 31.5s	remaining: 0us


In [182]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# predv = model.predict(te)
# rmse = (np.sqrt(mean_squared_error(tey, predv)))
# r2 = r2_score(tey, predv)
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))
# pred = polyreg.predict(te_data)

In [252]:
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# predv = model.predict(te)
# rmse = (np.sqrt(mean_squared_error(tey, predv)))
# r2 = r2_score(tey, predv)
# print('RMSE: {:.2f}'.format(rmse))
# print('R2: {:.2f}'.format(r2))
predt = model.predict(te_data)

In [256]:
sub=pd.DataFrame()
sub['ID']= id_
sub['Time_taken (min)'] = 0.9*predtlgb + 0.1*predt
sub.to_csv(r"C:\Users\admin\Downloads\dataset\hellosubcatlgb_wh.csv",index=False)